In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
N = 30
P = 5
X = pd.DataFrame({i:pd.Series(np.random.randint(P, size = N), dtype="category") for i in range(5)})
y = pd.Series(np.random.randint(P, size = N),  dtype="category")

In [3]:
X.head()

,0,1,2,3,4
0,0,1,3,1,0
1,3,1,4,2,1
2,1,0,1,1,1
3,0,2,2,3,0
4,4,0,3,1,1


In [4]:
y.head()

0    0
1    4
2    2
3    1
4    3
dtype: category
Categories (5, int64): [0, 1, 2, 3, 4]

In [5]:
def entropy(Y):
    """
    Function to calculate the entropy 

    Inputs:
    > Y: pd.Series of Labels
    Outpus:
    > Returns the entropy as a float
    """
    dict={}
    n=Y.size
    for i in Y:
        if i in dict:
            dict[i]+=1
        else:
            dict[i]=1
    ans=0
    for v in dict.values():
        frac=v/n
        ans+=-1*frac*math.log2(frac)

    return ans

In [6]:
def information_gain(Y, attr):
    """
    Function to calculate the information gain
    
    Inputs:
    > Y: pd.Series of Labels
    > attr: pd.Series of attribute at which the gain should be calculated
    Outputs:
    > Return the information gain as a float
    """
    total_entropy=entropy(Y)
    dict={}
    n=attr.size
    for i in attr.index:
        if attr[i] in dict:
            dict[attr[i]].append(Y[i])
        else:
            dict[attr[i]]=[Y[i]]
    temp=0
    for v in dict.values():
        x=pd.DataFrame(v)
        temp+=((len(v)/n)*entropy(x[0]))
    return (total_entropy-temp)
        

In [ ]:
np.random.seed(42)

class DecisionTree():
    def __init__(self, criterion, max_depth=5):
        """
        Put all infromation to initialize your tree here.
        Inputs:
        > criterion : {"information_gain", "gini_index"} # criterion won't be used for regression
        > max_depth : The maximum depth the tree can grow to 
        """
        self.crt=criterion
        self.max_dt=max_depth
        
        self.val=-1
        self.children=[]
        self.parent=None
        pass
    
    def get_atrb(self,X,y):        # still pending when two min_info_gains match (random pick)
        atrb=1000000
        info_gain=-10000000
        for i in X:
            temp=information_gain(y,X[i])
            if(info_gain<temp):
                info_gain=temp
                atrb=i
        return atrb    
            

    def fit(self, X, y):
        """
        Function to train and construct the decision tree
        Inputs:
        X: pd.DataFrame with rows as samples and columns as features (shape of X is N X P) where N is the number of samples and P is the number of columns.
        y: pd.Series with rows corresponding to output variable (shape of Y is N)
        """
        root=DecisionTree(self.crt,self.max_dt)
        if(X.size==0):
            root.val=y.mode()
            return root
        a=self.get_atrb(X,y)    #best_attribute
        root.val=a
        lt=X[a].unique()
#         print(root.val,end=" ")
        for i in lt:
            new_x=X.loc[X[a]==i]
            new_x=new_x.drop(a,axis=1)
            # y kelea
            indices=X.index[X[a]==i].tolist()
            new_y=y[indices]
            new_y.to_frame()
            
#             child=DecisionTree(self.crt,self.max_dt-1)
            
            if(new_x.size==0):
                child=DecisionTree(self.crt,self.max_dt-1)
                child.parent=root
                child.val=new_y.mode()
                return child
            child=root.fit(new_x,new_y)
            child.parent=root
            root.children.append(child);
        
        return root
        pass

    def predict(self, X):
        """
        Funtion to run the decision tree on a data point
        Input:
        X: pd.DataFrame with rows as samples and columns as features
        Output:
        y: pd.Series with rows corresponding to output variable. THe output variable in a row is the prediction for sample in corresponding row in X.
        """
        
        pass
#     def printer(self,root):
#         q=[]
#         q.append(root)
#         while(len(q)!=0):
#             a=q[0]
#             print(a.val)
#             q.pop(0)
#             for i in a.children:
#                 q.append(i)
    def get_level(self):
        l=0
        while self.parent:
            l+=1
        return l
            
    def plot(self):
        """
        Function to plot the tree

        Output Example:
        ?(X1 > 4)
            Y: ?(X2 > 7)
                Y: Class A
                N: Class B
            N: Class C
        Where Y => Yes and N => No
        """
#         spaces=self.get_level()*4*" "
#         prefix=spaces+"|__" if self.parent else ""
#         print(prefix+str(self.val))
        for i in self.children:
            i.plot()
        pass


In [ ]:
tree=DecisionTree("Entropy")
a=tree.fit(X,y)
a.plot()